# Magic callbacks

**Goal: Keep the analysis easy to read while also providing an input form that waits for a button click**

What that means:

  * Sequential logic that we can read top to bottom
  * Variables at global scope (easy to inspect/debug)
  * Multiple cells with intermediate outputs
  
But we also want:

  * User-friendly input form with widgets
  * "Run all friendliness" such that we can **wait** until the user clicks Run.  That enables us to do input validation before running the analysis.
  
Alternative solutions for executing cells programmatically:

  * The Jupyter Javascript API can do this ([discussion](https://github.com/jupyter/notebook/issues/2660)) but you have to know the cell number (fragile) and it doesn't work in Voila.  Example at the bottom of this notebook.
  * If you hardcode the notebook filename, you can use nbformat (etc.) to open it and run cells (again, knowing which ones you want to run).
  
Note: [callable_cells.ipynb](callable_cells.ipynb) in this directory is probably a better/cleaner way to implement the magics.

## The magic parts

This stuff would be wrapped up into a library...

In [ ]:
from IPython.core.magic import register_cell_magic

# Dictionary of cells that we want to be able to run as callbacks
callbacks = {}

# Return a function that runs the code in one or more cells
def run_callbacks(*names):
    def the_callback(w):
        output.clear_output()
        with output:
            for name in names:
                exec(callbacks[name], globals(), globals())
    return the_callback

# Register a cell as being able to run as a callback
@register_cell_magic
def callback(line, cell):
    # Store the cell's code
    callbacks[line] = cell
    # Also run the code now, unless it has a special name
    if line != 'inputs':
        exec(cell, globals(), globals())

## My awesome scientific analysis

Sequential logic, global scope, intermediate outputs, easy to debug.

Cells that we want to run from the dashboard's input form are annotated with `%%callback` names.  Ideally the dashboard would suppress outputs from this section the first time it runs (see [voila issue #171](https://github.com/QuantStack/voila/issues/171)).

In [ ]:
# Set an initial value while I'm doing exploratory analysis

x = 4

In [ ]:
%%callback square

print(f"square = {x*x}")

In [ ]:
# Extra stuff I don't want in the final dashboard, so no %%callback annotation

print(f"x is {'odd' if x % 2 else 'even'}")

In [ ]:
%%callback sqrt

import math
print(f"root = {math.sqrt(x)}")

## Widget input form

After user enters parameters and clicks Run, validate inputs, then execute the analysis above.

All outputs from executing the code above get redirected into an Output widget below the Button.

If I'm still in Jupyter notebook (not dashboard), and I get an error, it's easy to debug because everything is still global -- I can just add a new cell and print things out.

In [ ]:
import ipywidgets as widgets

text = widgets.Text(description='x: ')
button = widgets.Button(description='Run')
button.on_click(run_callbacks('inputs', 'square', 'sqrt'))
output = widgets.Output()
widgets.VBox([text, button, output])

In [ ]:
%%callback inputs

if text.value:
    x = int(text.value)
else:
    raise RuntimeError('must set x!')

#### Javascript version

For reference.  Not ideal + doesn't work in voila.

In [ ]:
from IPython.display import display, Javascript
button2 = widgets.Button(description='JS Run')
def run_cells(w):
    with output2:
        display(Javascript('IPython.notebook.execute_cells([5, 7])'))
button2.on_click(run_cells)
output2 = widgets.Output()
widgets.VBox([button2, output2])